Será que esse modelo de árvore de decisão é nossa melhor escolha? Podemos optar por usar não apenas uma árvore, mas uma floresta inteira, o que geralmente dá melhores resultados. Vamos fazer isso.

Vamos aproveitar e dar uma organizada no nosso código. Primeiro, todos os imports que fizemos no esquenta.

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import mean_absolute_error

Então carregamos os arquivos de treino e teste.

In [0]:
dados_treino = pd.read_csv('treino.csv', index_col='Id')
dados_teste = pd.read_csv('teste.csv', index_col='Id')

Escolhemos as características.

In [0]:
caracteristicas = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 
                   'FullBath', 'BedroomAbvGr','TotRmsAbvGrd']

Fazemos a copia dos dados.

In [0]:
X = dados_treino[caracteristicas].copy()
y = dados_treino.SalePrice

Dividimos os nossos dados entre treino e validação.

In [0]:
X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, 
                                                    train_size=0.8, 
                                                    test_size=0.2, 
                                                    random_state=42)

Temos o nosso modelo simples de árvore. Vamos fazer um com a floresta agora.

In [0]:
modelo_1 = tree.DecisionTreeRegressor(random_state=42)

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
modelo_2 = RandomForestRegressor(random_state=42)

Vamos ver como está o erro desses modelos.

In [14]:
modelo_1.fit(X_treino, y_treino)
preds_1 = modelo_1.predict(X_valid)
mae_1 = mean_absolute_error(y_valid, preds_1)
mae_1

30821.260273972603

In [15]:
modelo_2.fit(X_treino, y_treino)
preds_2 = modelo_2.predict(X_valid)
mae_2 = mean_absolute_error(y_valid, preds_2)
mae_2

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


22740.137181996088

Podemos mostrar os erros de uma maneira organizada, e repare que o modelo da floresta, como esperado, tem um erro menor.

In [16]:
print('MAE do modelo de árvore: %d'% mae_1)
print('MAE do modelo de floresta: %d'% mae_2)

MAE do modelo de árvore: 30821
MAE do modelo de floresta: 22740


Será que esse erro pode ser menor ainda? Como podemos fazer isso? Podemos alterar o que passamos para o nosso modelo e escolhermos um modelo com o menor resultado de erro.

In [0]:
modelo_3 = RandomForestRegressor(n_estimators=50, random_state=42)
modelo_4 = RandomForestRegressor(n_estimators=100, random_state=42)
modelo_5 = RandomForestRegressor(n_estimators=100, criterion='mae', random_state=42)
modelo_6 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=42)
modelo_7 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=42)

Repare que definimos mais cinco modelo de floresta com parâmetros diferentes, agora para o nosso código ficar menos repetitivos, podemos agrupa-los em uma lista.

In [0]:
modelos = [modelo_3, modelo_4, modelo_5, modelo_6, modelo_7]

Agora, para cada modelo na nossa lista de modelos, podemos fazer o fit, predict e imprimir o mae.

In [27]:
for i in range(0, len(modelos)):
  modelos[i].fit(X_treino, y_treino)
  preds = modelos[i].predict(X_valid)
  mae = mean_absolute_error(y_valid, preds)
  print('MAE do modelo_%d: %d' % (i+3, mae))

MAE do modelo_3: 22411
MAE do modelo_4: 22537
MAE do modelo_5: 22569
MAE do modelo_6: 22719
MAE do modelo_7: 23044


Repare que o melhor modelo que temos é o modelo_3

In [0]:
melhor_modelo = modelo_3

Agora que sabemos qual é um melhor modelo, podemos predizer os resultados com ele e salvá-los.
Para isso, vamos definir X_teste.

In [0]:
X_teste = dados_teste[caracteristicas].copy()

In [0]:
melhor_modelo.fit(X,y)
predicoes_teste = melhor_modelo.predict(X_teste)

Vamos montar nosso dicionário e salvá-lo em um arquivo.

In [0]:
resultados_floresta = {'Id': X_teste.index, 'SalePrice': predicoes_teste}

In [0]:
resultados = pd.DataFrame(resultados_floresta)

In [0]:
resultados.to_csv('resultados_floresta.csv', index=False)